<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [7]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, launch_site in launch_sites_df.iterrows():
  coordinate = [launch_site['Lat'], launch_site['Long']]
  circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(launch_site['Launch Site']))
  marker = folium.map.Marker(coordinate,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site['Launch Site'], ))

  site_map.add_child(circle)
  site_map.add_child(marker)

site_map

Launch sites tend to go close to equator and next to coastal lines.

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [10]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [11]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
success_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, record in spacex_df.iterrows():
    #print(coordinate)
    coordinate = [record['Lat'], record['Long']]
    marker = folium.map.Marker(coordinate, icon=folium.Icon(icon_size=(20,20), color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

success_map.add_child(marker_cluster)
success_map

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
#East Coast coastal line
east_coast_coastline_lat = 28.564047
east_coast_coastline_lon = -80.568293

east_coast_distance_coastline = calculate_distance(launch_sites_df['Lat'][0], launch_sites_df['Long'][0],
  east_coast_coastline_lat, east_coast_coastline_lon)

east_coast_coordinate_coast = [east_coast_coastline_lat, east_coast_coastline_lon]
east_coast_distance_marker_coast = folium.Marker(
  east_coast_coordinate_coast,
  icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(east_coast_distance_coastline),))

east_coast_coordinates_coast = [(launch_sites_df['Lat'][0], launch_sites_df['Long'][0]), east_coast_coordinate_coast]
east_coast_lines = folium.PolyLine(locations=east_coast_coordinates_coast, weight=2)
success_map.add_child(east_coast_lines)
success_map.add_child(east_coast_distance_marker_coast)

In [16]:
#West Coast coastal line
west_coast_coastline_lat = 34.633825
west_coast_coastline_lon = -120.625791

west_coast_distance_coastline = calculate_distance(launch_sites_df['Lat'][3], launch_sites_df['Long'][3],
  west_coast_coastline_lat, west_coast_coastline_lon)

west_coast_coordinate_coast = [west_coast_coastline_lat, west_coast_coastline_lon]
west_coast_distance_marker_coast = folium.Marker(
  west_coast_coordinate_coast,
  icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(west_coast_distance_coastline),))

west_coast_coordinates_coast = [(launch_sites_df['Lat'][3], launch_sites_df['Long'][3]), west_coast_coordinate_coast]
west_coast_lines = folium.PolyLine(locations=west_coast_coordinates_coast, weight=2)
success_map.add_child(west_coast_lines)
success_map.add_child(west_coast_distance_marker_coast)

In [17]:
#Equator distance to sites
east_coast_equator_lat = 0
east_coast_equator_lon = -80.576820
west_coast_equator_lat = 0
west_coast_equator_lon = -120.485472

east_coast_distance_equator = calculate_distance(launch_sites_df['Lat'][0], launch_sites_df['Long'][0],
  east_coast_equator_lat, east_coast_equator_lon)
west_coast_distance_equator = calculate_distance(launch_sites_df['Lat'][3], launch_sites_df['Long'][3],
  west_coast_equator_lat, west_coast_equator_lon)

east_coast_coordinate_equator = [east_coast_equator_lat, east_coast_equator_lon]
west_coast_coordinate_equator = [west_coast_equator_lat, west_coast_equator_lon]

east_coast_distance_marker_equator = folium.Marker(
  east_coast_coordinate_equator, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(east_coast_distance_equator),))
west_coast_distance_marker_equator = folium.Marker(
  west_coast_coordinate_equator, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(west_coast_distance_equator),))

east_coast_coordinates_equator = [(launch_sites_df['Lat'][0], launch_sites_df['Long'][0]), east_coast_coordinate_equator]
east_coast_lines_equator = folium.PolyLine(locations=east_coast_coordinates_equator, weight=3)
west_coast_coordinates_equator = [(launch_sites_df['Lat'][3], launch_sites_df['Long'][3]), west_coast_coordinate_equator]
west_coast_lines_equator = folium.PolyLine(locations=west_coast_coordinates_equator, weight=3)

success_map.add_child(east_coast_lines_equator)
success_map.add_child(east_coast_distance_marker_equator)
success_map.add_child(west_coast_lines_equator)
success_map.add_child(west_coast_distance_marker_equator)

The distance from the equator from the sites is between 3100 km and 3900 km

In [18]:
#Launch sites in close proximity to railways and highways
marker_cluster = MarkerCluster()
success_map_railways_highways = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker = folium.map.Marker(coordinate, icon=folium.Icon(icon_size=(20,20), color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

success_map_railways_highways.add_child(marker_cluster)

#east coast railways & highways
coordinates = {
    "nasa_railroad": {"lat": 28.572102, "lon": -80.585277},
    "i95": {"lat": 28.567339, "lon": -80.851895},
    "i4": {"lat": 28.558736, "lon": -81.380533}
}

east_coast_distance_poi = []
east_coast_distance_marker_poi = []

for poi, coordinate in coordinates.items():
    distance = calculate_distance(launch_sites_df['Lat'][0], launch_sites_df['Long'][0], coordinate["lat"], coordinate["lon"])
    east_coast_distance_poi.append(distance)
    marker_html = '<div style="font-size: 48; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    east_coast_distance_marker_poi.append(folium.Marker((coordinate["lat"], coordinate["lon"]), icon=DivIcon(icon_size=(20,20),
    icon_anchor=(0,0), html=marker_html)))

east_coast_coordinates_poi = [
    [(launch_sites_df['Lat'][0], launch_sites_df['Long'][0]), (coordinates["lat"], coordinates["lon"])]
    for coordinates in coordinates.values()]

east_coast_poi_to_site_lines = [folium.PolyLine(locations=poi, weight=3) for poi in east_coast_coordinates_poi]

for line in east_coast_poi_to_site_lines:
    success_map_railways_highways.add_child(line)

for marker in east_coast_distance_marker_poi:
    success_map_railways_highways.add_child(marker)

#west coast railways & highways
coordinates = {
    "union_pacific": {"lat": 34.633414, "lon": -120.624806},
    "ca1": {"lat": 34.765000, "lon": -120.301932}
}

west_coast_distance_poi = []
west_coast_distance_marker_poi = []

for poi, coordinate in coordinates.items():
    distance = calculate_distance(launch_sites_df['Lat'][3], launch_sites_df['Long'][3], coordinate["lat"], coordinate["lon"])
    west_coast_distance_poi.append(distance)
    marker_html = '<div style="font-size: 48; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    west_coast_distance_marker_poi.append(folium.Marker((coordinate["lat"], coordinate["lon"]), icon=DivIcon(icon_size=(20,20),
    icon_anchor=(0,0), html=marker_html)))

west_coast_coordinates_poi = [
    [(launch_sites_df['Lat'][3], launch_sites_df['Long'][3]), (coordinates["lat"], coordinates["lon"])]
    for coordinates in coordinates.values()]

west_coast_poi_to_site_lines = [folium.PolyLine(locations=poi, weight=3) for poi in west_coast_coordinates_poi]

for line in west_coast_poi_to_site_lines:
    success_map_railways_highways.add_child(line)

for marker in west_coast_distance_marker_poi:
    success_map_railways_highways.add_child(marker)

success_map_railways_highways


#### The 3 Points of interest for the east coast sites are the I4, I95 and the Nasa Railroad.
##### Distance to Nasa Railroad is 1.34 km.
##### Distance to i95 is 26.83 km.
##### Distance to i4 is 78.47 km.

#### The 2 Points of interest for the west coast sites are the union pacific railroad and the CA 1 highway.
##### Distance UP railroad is 1.29 km.
##### Distance to CA 1 is 31.84 km.

In [19]:
marker_cluster = MarkerCluster()
success_map_cities = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker = folium.map.Marker(coordinate, icon=folium.Icon(icon_size=(20,20), color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

success_map_cities.add_child(marker_cluster)

# East Coast cities
city_coordinates = {
    "miami": {"lat": 25.78, "lon": -80.21},
    "tampa": {"lat": 27.94, "lon": -82.45},
    "jax": {"lat": 30.33, "lon": -81.66},
    "orl": {"lat": 28.54, "lon": -81.38}
}

east_coast_distance_cities = []
east_coast_distance_marker_cities = []

for city, coordinates in city_coordinates.items():
    distance = calculate_distance(launch_sites_df['Lat'][0], launch_sites_df['Long'][0], coordinates["lat"], coordinates["lon"])
    east_coast_distance_cities.append(distance)
    marker_html = '<div style="font-size: 24; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    east_coast_distance_marker_cities.append(folium.Marker((coordinates["lat"], coordinates["lon"]), icon=DivIcon(icon_size=(20,20),
    icon_anchor=(0,0), html=marker_html)))

east_coast_coordinates_cities = [
    [(launch_sites_df['Lat'][0], launch_sites_df['Long'][0]), (coordinates["lat"], coordinates["lon"])]
    for coordinates in city_coordinates.values()]

east_coast_city_to_site_lines = [folium.PolyLine(locations=city, weight=3) for city in east_coast_coordinates_cities]

for line in east_coast_city_to_site_lines:
    success_map_cities.add_child(line)

for marker in east_coast_distance_marker_cities:
    success_map_cities.add_child(marker)

# West Coast cities
city_coordinates = {
    "lax": {"lat": 34.05, "lon": -118.25},
    "sfo": {"lat": 37.77, "lon": -122.41},
    "vegas": {"lat": 36.17, "lon": -115.13}
}

west_coast_distance_cities = []
west_coast_distance_marker_cities = []

for city, coordinates in city_coordinates.items():
    distance = calculate_distance(launch_sites_df['Lat'][3], launch_sites_df['Long'][3], coordinates["lat"], coordinates["lon"])
    west_coast_distance_cities.append(distance)
    marker_html = '<div style="font-size: 24; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    west_coast_distance_marker_cities.append(folium.Marker((coordinates["lat"], coordinates["lon"]), icon=DivIcon(icon_size=(20,20),
    icon_anchor=(0,0), html=marker_html)))

west_coast_coordinates_cities = [
    [(launch_sites_df['Lat'][3], launch_sites_df['Long'][3]), (coordinates["lat"], coordinates["lon"])]
    for coordinates in city_coordinates.values()]

west_coast_city_to_site_lines = [folium.PolyLine(locations=city, weight=3) for city in west_coast_coordinates_cities]

for line in west_coast_city_to_site_lines:
    success_map_cities.add_child(line)

for marker in west_coast_distance_marker_cities:
    success_map_cities.add_child(marker)

success_map_cities

#### The 4 cities we chosen for the east coast sites are Miami, Orlando, Tampa Bay and Jacksonville.
##### Distance to Miami is 311 km.
##### Distance to Orlando is 78 km.
##### Distance to Tampa Bay is 196 km.
##### Distance to Jacksonville is 222 km.

#### The 3 cities we chosen for the west coast sites are Los Angeles, San Francisco and Las Vegas.
##### Distance to Los Angales 226 km.
##### Distance to San Francisco is 384 km.
##### Distance to Las Vegas 525 km.

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
